In [1]:
# 1. Import Dash
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

import pandas as pd
import plotly.express as px

# --- DAHSBOARD ELEMENTS ---

# DATASET
gpp = pd.read_csv('data_input/power_plant.csv')

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Home", href="#")),
    ],
    brand="Global Power Plant",
    brand_href="#",
    color="#696969",
    dark=True,
    sticky='top'
)

# CARDS

total_country = [
    dbc.CardHeader('Number of Country'),
    dbc.CardBody(
        [
            html.H1(gpp['country_long'].unique().shape[0])
        ]
    )
]

total_power_plant = [
    dbc.CardHeader('Total Power Plant'),
    dbc.CardBody(
        [
            html.H1(gpp['name of powerplant'].unique().shape[0])
        ]
    )
]

fuel_type = [
    dbc.CardHeader('Total Type of Fuel'),
    dbc.CardBody(
        [
            html.H1(gpp['primary_fuel'].unique().shape[0])
        ]
    )
]


# GRAPH

###### --- CHOROPLETH
agg1 = pd.crosstab(
    index=[gpp['country code'], gpp['start_year']],
    columns='No of Power Plant'
).reset_index()

map = px.choropleth(agg1.sort_values("start_year"),
                    locations='country code',
                    color_continuous_scale='tealgrn',
                    color='No of Power Plant',
                    animation_frame='start_year',
                    template='ggplot2')

# 2. Create a Dash app instance
app = dash.Dash(
    name='Global Power Plant',
    external_stylesheets=[dbc.themes.LUX]
)
app.title = 'Global Power Plant Dashboard'

# --- DASHBOARD LAYOUT

app.layout = html.Div(children=[
    navbar,
    html.Br(),

    # Main page
    html.Div(
        [
            # ----- ROW 1
            dbc.Row(
                [
                    # --- COLUMN 1
                    dbc.Col(
                        [
                            dbc.Card(
                                total_country, color='aquamarine',

                            ),
                            html.Br(),
                            dbc.Card(
                                total_power_plant, color='plum',
                            ),
                            html.Br(),
                            dbc.Card(
                                fuel_type, color='lightsalmon',
                            ),
                        ],
                        width=3,
                    ),

                    # --- COLUMN 2
                    dbc.Col(
                        [
                            dcc.Graph(
                                id='choropleth',
                                figure=map
                            )
                        ],
                        width=9
                    ),
                ]
            ),

            html.Hr(),

            # ----- ROW 2
            dbc.Row(
                [
                    # --- COLUMN 1
                    dbc.Col(
                        [
                            html.H1(
                                'Analysis by Country',
                            ),
                            dbc.Tabs(
                                [
                                    dbc.Tab(
                                        dcc.Graph(
                                            id='plot1_overall',
                                        ),
                                        label='Ranking'
                                    ),
                                    dbc.Tab(
                                        dcc.Graph(
                                            id='plot3_capacity',
                                        ),
                                        label='Distribution'
                                    ),
                                ]
                            ),
                        ],
                        width=8,
                    ),

                    # --- COLUMN 2
                    dbc.Col(
                        [
                            dbc.Card(
                                [
                                    dbc.CardHeader('Select Country'),
                                    dbc.CardBody(
                                        dcc.Dropdown(
                                            id='list_country',
                                            options=gpp['country_long'].unique(
                                            ),
                                            value='Indonesia',
                                        ),
                                    ),
                                ]
                            ),
                            dcc.Graph(
                                id='plot2_fuel',
                            ),
                        ],
                        width=4,
                    ),

                ]
            ),

        ],
        style={
            'padding-right': '30px',
            'padding-left': '30px'
        }
    )


],
    style={
    'backgroundColor': 'seashell',
},
)

# ---- CALLBACK TO SELECT COUNTRY

# ---- PLOT 1: Barplot


@app.callback(
    Output(component_id='plot1_overall', component_property='figure'),
    Input(component_id='list_country', component_property='value'),
)
def update_plot1(country_name):
    # --- BAR CHART
    # Data aggregation
    gpp_indo = gpp[gpp['country_long'] == country_name]
    top_indo = gpp_indo.sort_values('capacity in MW').tail(10)

    # Visualize
    plot1_bar = px.bar(
        top_indo,
        x='capacity in MW',
        y='name of powerplant',
        template='ggplot2',
        title=f'Rangking of Overall Power Plants in {str(country_name)}'
    )

    return plot1_bar

# ---- PLOT 2: Pie Chart


@app.callback(
    Output(component_id='plot2_fuel', component_property='figure'),
    Input(component_id='list_country', component_property='value'),
)
def update_plot2(country_name):
    # --- PIE CHART
    # aggregation
    gpp_indo = gpp[gpp['country_long'] == country_name]

    agg2 = pd.crosstab(
        index=gpp_indo['primary_fuel'],
        columns='No of Power Plant'
    ).reset_index()

    # visualize
    plot2_pie = px.pie(
        agg2,
        values='No of Power Plant',
        names='primary_fuel',
        color_discrete_sequence=['aquamarine',
                                 'salmon', 'plum', 'grey', 'slateblue'],
        template='ggplot2',
        hole=0.4,
        labels={
            'primary_fuel': 'Type of Fuel'
        }
    )
    return plot2_pie

# ---- PLOT 3: Box Plot


@app.callback(
    Output(component_id='plot3_capacity', component_property='figure'),
    Input(component_id='list_country', component_property='value'),
)
def update_plot2(country_name):
    gpp_indo = gpp[gpp['country_long'] == country_name]
    ###### --- BOXPLOT
    gpp_capacity = gpp_indo[gpp_indo['capacity in MW'] < 3000]
    plot3_box = px.box(
        gpp_capacity,
        color='primary_fuel',
        y='capacity in MW',
        template='ggplot2',
        title=f'Distribution of capacity in MW in each fuel in {str(country_name)}',
        labels={
            'primary_fuel': 'Type of Fuel'
        }
    )
    return plot3_box


# 3. Start the Dash server
if __name__ == "__main__":
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'Global Power Plant'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_2m1667280642.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash/deps/react@16.v2_6_2m1667280642.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_2m1667280642.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_2m1667280642.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1667280676.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54:36] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_2m1667280642.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 07:54